# Clasificadores

### Introducción:



En esta clase, nos enfocaremos en algunos de los algoritmos más poderosos y populares para la clasificación, como **Support Vector Machines (SVM)**, **Árboles de Decisión**, **Random Forest**, y los algoritmos avanzados de boosting como **XGBoost** y **LightGBM**. Cada uno de estos métodos tiene sus propias fortalezas, debilidades y áreas de aplicación, por lo que es crucial entender no solo cómo funcionan, sino también cuándo y por qué elegir uno sobre otro.

A lo largo de la clase, exploraremos:
1. **SVM**, que es particularmente útil cuando trabajamos con datos de alta dimensionalidad y necesitamos encontrar la mejor frontera de decisión.
2. **Árboles de Decisión**, que nos permiten tomar decisiones claras y visualmente intuitivas, pero que pueden ser propensos al sobreajuste.
3. **Random Forest**, que mejora la estabilidad y precisión de los árboles de decisión al combinar múltiples árboles para generar predicciones robustas.
4. **XGBoost** y **LightGBM**, que utilizan técnicas avanzadas de boosting para corregir iterativamente los errores de los modelos previos y lograr resultados altamente precisos en problemas complejos.

Al final de esta clase, tendrán una comprensión clara de cómo usar estos métodos en sus propios proyectos de machine learning y cómo interpretar los resultados obtenidos.

# SVM

El **SVM** busca encontrar un hiperplano que maximice el margen entre las clases en un espacio de características, donde solo los vectores de soporte (los puntos más cercanos al hiperplano) definen su posición y orientación.

Para trabajar con datos no linealmente separables, **SVM** usa funciones de kernel como el lineal, polinómico o RBF, lo que permite operar en espacios de características más altos sin calcular explícitamente sus coordenadas. El ajuste de los hiperparámetros como **C** (que controla el margen y los errores de clasificación) y **Gamma** (que influye en la complejidad de los límites de decisión) es clave para optimizar el modelo.

Las ventajas de **SVM** incluyen su eficacia en espacios de alta dimensión y su capacidad de manejar datos no lineales mediante el uso de kernels. Sin embargo, puede ser ineficiente para conjuntos de datos muy grandes y requiere ajustes cuidadosos del kernel y los parámetros.




In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import datasets
import pandas as pd
import numpy as np
import warnings

# Cargar el conjunto de datos iris
iris = datasets.load_iris()



# Extraer las características (columnas) del dataset Iris
X = pd.DataFrame(iris["data"], columns=iris["feature_names"])


# Crear el vector objetivo (1 si es Iris-Virginica, de lo contrario 0)
y = iris["target"]   # 0 para Iris-Setosa, 1 para Iris-Versicolor, 2 para Iris-Virginica


# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

# Normalizar los datos: utilizar la media y desviación estándar del conjunto de entrenamiento
mean_X_train = np.mean(X_train, axis=0)
std_X_train = np.std(X_train, axis=0)

X_train_z = (X_train - mean_X_train) / std_X_train  # Normalización de entrenamiento
X_test_z = (X_test - mean_X_train) / std_X_train    # Normalización de prueba con estadísticas de entrenamiento

X_train_z=X_train_z[['petal length (cm)','petal width (cm)']]
X_test_z=X_test_z[['petal length (cm)','petal width (cm)']]





Un hiperparametro importante para el modelo SVM es el  **hiperparámetro $C$** el cual controla el equilibrio entre la maximización del margen y la minimización de los errores de clasificación. Es esencial en los **Soft Margin SVM**, donde los datos pueden no ser separables linealmente, permitiendo que algunos puntos caigan dentro del margen o se clasifiquen incorrectamente.

### Función de $C$:
El parámetro $C$ establece la importancia que le damos a los **errores de clasificación** en comparación con el **tamaño del margen**. Se ajusta en la función objetivo del modelo, la cual intenta minimizar tanto el tamaño del margen como los errores cometidos por el modelo. Matemáticamente, la función objetivo es:

$$
\frac{1}{2} ||w||^2 + C \sum_{i=1}^{n} \xi_i
$$

Donde:
- $||w||^2$ está inversamente relacionado con el margen en **SVM**: un margen pequeño implica un valor grande de $||w||^2$, y un margen grande implica un valor pequeño. Minimizar $||w||^2$ maximiza el margen.
- $\xi_i$ son las **variables de holgura** que permiten errores dentro del margen.
- $C$ es un hiperparámetro que equilibra un margen grande con la cantidad de errores permitidos.

### Efecto del parámetro $C$:
1. **$C$ grande**:
   - El modelo prioriza **minimizar los errores** de clasificación.
   - El margen será más estrecho, permitiendo menos puntos dentro del margen o mal clasificados.
   - En este caso, el modelo se vuelve más **estricto** y tiende a ser **menos tolerante a errores**.
   - Sin embargo, al ser más estricto, también puede sobreajustar los datos (overfitting), lo que significa que el modelo puede ajustarse demasiado a los datos de entrenamiento, pero podría no generalizar bien a los datos de prueba.

2. **$C$ pequeño**:
   - El modelo prioriza **maximizar el margen** sobre la reducción de errores.
   - Permite más errores de clasificación, ya que los puntos que están mal clasificados o dentro del margen reciben menos penalización.
   - Un $C$ más pequeño hace que el modelo sea **más flexible** y puede evitar el sobreajuste, resultando en un mejor ajuste a los datos de prueba (mejor generalización).
   - Sin embargo, un $C$ muy pequeño puede hacer que el modelo subajuste (underfitting) y no capture correctamente las relaciones en los datos de entrenamiento.

### Visualización:
- **$C$ grande**: El modelo intentará ajustar casi todos los puntos de datos correctamente, incluso si eso significa que el margen será más pequeño.


- **$C$ pequeño**: El modelo tolerará más errores y buscará un margen más amplio, aunque algunos puntos queden dentro del margen o mal clasificados.
  




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.datasets import make_classification

# Crear un conjunto de datos simple para clasificación binaria
X, y = make_classification(n_samples=100, n_features=2, n_informative=2, n_redundant=0, random_state=42)

# Crear dos clasificadores SVM con diferentes valores de C
clf_high_C = svm.SVC(kernel='linear', C=100)
clf_low_C = svm.SVC(kernel='linear', C=0.01)

# Ajustar los modelos
clf_high_C.fit(X, y)
clf_low_C.fit(X, y)

# Crear un grid para las visualizaciones
xx, yy = np.meshgrid(np.linspace(X[:, 0].min()-1, X[:, 0].max()+1, 100),
                     np.linspace(X[:, 1].min()-1, X[:, 1].max()+1, 100))

# Predicción en el grid para obtener las fronteras de decisión
Z_high_C = clf_high_C.decision_function(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
Z_low_C = clf_low_C.decision_function(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

# Visualización de los modelos
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# Gráfico para C grande (ajuste más estricto)
ax1.contourf(xx, yy, Z_high_C, levels=np.linspace(Z_high_C.min(), 0, 7), cmap='coolwarm', alpha=0.5)
ax1.contour(xx, yy, Z_high_C, levels=[0], linewidths=2, colors='black')
ax1.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap='coolwarm', edgecolors='k')
ax1.set_title('SVM con C alto (C=100)')

# Gráfico para C pequeño (margen más amplio)
ax2.contourf(xx, yy, Z_low_C, levels=np.linspace(Z_low_C.min(), 0, 7), cmap='coolwarm', alpha=0.5)
ax2.contour(xx, yy, Z_low_C, levels=[0], linewidths=2, colors='black')
ax2.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap='coolwarm', edgecolors='k')
ax2.set_title('SVM con C bajo (C=0.01)')

plt.show()


Ahora teniendo los conceptos básicos de SVM, se va aplicar a nuestro problema del conjunto iris

In [ ]:
from sklearn.svm import SVC
svm=SVC(kernel='linear',C=1,random_state=1)
svm.fit(X_train_z,y_train)



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

# Función para graficar las regiones de decisión
def plot_decision_region(clf, X, y, test_idx=None, resolution=0.02):
    # Configurar los colores del mapa y los puntos
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # Seleccionar dos características para visualizar
    X = X.iloc[:, :2].values  # Solo las dos primeras columnas para graficar en 2D

    # Crear los límites del gráfico
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))

    # Obtener las predicciones para la cuadrícula
    Z = clf.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)

    # Graficar la superficie de decisión
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    # Graficar los puntos de entrenamiento
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],
                    alpha=0.8, c=colors[idx],
                    marker=markers[idx], label=f'Clase {cl}', edgecolor='black')

    plt.xlabel('Primera característica')
    plt.ylabel('Segunda característica')
    plt.legend(loc='upper left')
    plt.show()

# Entrenar un SVM con solo dos características para visualizar
from sklearn.svm import SVC

# Utilizar solo las primeras dos características para la visualización
X_train_2D = X_train_z.iloc[:, :2]  # Seleccionamos solo las primeras 2 columnas
y_train_2D = y_train

# Definir el clasificador SVM
svm_clf = SVC(kernel='linear', C=1.0, random_state=1)
svm_clf.fit(X_train_2D, y_train_2D)

# Graficar las regiones de decisión
plot_decision_region(svm_clf, X_train_2D, y_train_2D)


### Solución de problemas no lineales

Con **SVM** también es posible resolver problemas no lineales mediante el uso de **kernels**. Al cambiar el kernel, podemos seleccionar entre diferentes tipos como **linear**, **poly** (polinómico), **rbf** (radial), y **sigmoid**. A continuación, veremos un ejemplo sintético que demuestra cómo funciona este proceso.

In [ ]:
from sklearn.datasets import make_circles
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import numpy as np
# Crear un conjunto de datos sintético
X, y = make_circles(n_samples=300, factor=0.5, noise=0.1, random_state=42)

# Visualizar los datos
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm')
plt.title('Datos Simulados')
plt.xlabel('Característica 1')
plt.ylabel('Característica 2')
plt.show()


In [ ]:
# Escalar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Definir los modelos con diferentes kernels
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
models = [SVC(kernel=kernel, gamma='scale',C=2).fit(X_scaled, y) for kernel in kernels]

# Crear una malla para visualizar los límites de decisión
xx, yy = np.meshgrid(np.linspace(X_scaled[:, 0].min() - 1, X_scaled[:, 0].max() + 1, 500),
                     np.linspace(X_scaled[:, 1].min() - 1, X_scaled[:, 1].max() + 1, 500))
grid = np.c_[xx.ravel(), yy.ravel()]

# Visualizar los límites de decisión
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
titles = ['Kernel Lineal', 'Kernel Polinómico', 'Kernel RBF', 'Kernel Sigmoide']

for model, kernel, ax, title in zip(models, kernels, axes.ravel(), titles):
    Z = model.decision_function(grid).reshape(xx.shape)
    ax.contourf(xx, yy, Z, levels=np.linspace(Z.min(), 0, 7), cmap='coolwarm', alpha=0.5)
    ax.contour(xx, yy, Z, levels=[0], linewidths=2, colors='black')
    ax.scatter(X_scaled[:, 0], X_scaled[:, 1], c=y, cmap='coolwarm', edgecolors='k')
    ax.set_title(title)
    ax.set_xlabel('Característica 1')
    ax.set_ylabel('Característica 2')

plt.tight_layout()
plt.show()

## Árboles de decisiones

El **Árbol de Decisión** es un modelo de clasificación que crea una estructura jerárquica de decisiones, donde cada nodo interno representa una característica o atributo, y cada hoja representa una clase o etiqueta. El objetivo es dividir el espacio de características en regiones puras que maximicen la separación entre las clases. Los criterios comunes para elegir las divisiones son el **índice de Gini** o la **entropía**, que miden la impureza de las particiones. El proceso de división continúa hasta que las hojas contienen muestras homogéneas o hasta que se alcanza una profundidad máxima.

Las ventajas de los **Árboles de Decisión** incluyen su facilidad de interpretación y la capacidad de manejar tanto datos numéricos como categóricos. Sin embargo, tienden a sobreajustarse en los datos de entrenamiento si no se controla su profundidad. Para mitigar esto, es crucial ajustar hiperparámetros como la **profundidad máxima** o el **número mínimo de muestras por hoja**. Aunque son eficientes en conjuntos de datos pequeños a medianos, pueden volverse menos precisos en problemas más complejos o cuando los datos son ruidosos.



### Creación de un árbol de decisiones

Los árboles de decisión pueden construir **fronteras de decisión complejas** al dividir el espacio de características en regiones rectangulares o cúbicas (en espacios de mayor dimensión). Sin embargo, es importante tener cuidado, ya que **cuanto más profundo sea el árbol**, más específicas y detalladas serán las divisiones, lo que puede hacer que el modelo se ajuste demasiado a los datos de entrenamiento (sobreajuste), perdiendo capacidad de generalización.


In [ ]:
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import numpy as np

# Crear y ajustar el modelo del árbol de decisión
tree_model = DecisionTreeClassifier(criterion='gini', max_depth=4, random_state=1)
tree_model.fit(X_train_z.iloc[:, :2], y_train)  # Usamos solo las dos primeras características

# Definir la función de visualización mejorada de las regiones de decisión
def plot_decision_region(clf, X, y, resolution=0.02):
    # Crear una paleta de colores y marcar diferentes clases
    markers = ('o', 's', '^', 'v')
    colors = ('#FF9999', '#66B2FF', '#99FF99')
    cmap = plt.cm.RdYlBu

    # Seleccionar solo las primeras dos características para la visualización
    X = X.iloc[:, :2].values  # Convertir a numpy array para asegurar que sea manejable

    # Crear los límites de los ejes
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))

    # Predecir las etiquetas para cada punto de la malla
    Z = clf.predict(np.c_[xx1.ravel(), xx2.ravel()])
    Z = Z.reshape(xx1.shape)

    # Crear la superficie de decisión
    plt.contourf(xx1, xx2, Z, alpha=0.3, cmap=cmap)

    # Ajustar límites de los ejes
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    # Graficar los puntos de datos
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1], alpha=0.8, color=colors[idx],
                    edgecolor='k', marker=markers[idx], label=f'Clase {cl}', s=100)

# Llamar a la función mejorada para graficar
plot_decision_region(tree_model, X_train_z.iloc[:, :2], y_train)

# Configurar el título y las etiquetas del gráfico
plt.title('Árbol de Decisión - Regiones de Decisión')
plt.xlabel('Característica 1 (Estandarizada)')
plt.ylabel('Característica 2 (Estandarizada)')
plt.legend(loc='upper left')
plt.grid(True)
plt.show()


Una característica importante de los **árboles de decisión** es que nos permiten visualizar el proceso de toma de decisiones en cada nodo. Cada árbol puede ser analizado individualmente para entender cómo se dividen los datos y cómo se llegan a las predicciones, lo que facilita la interpretación de los patrones aprendidos por el modelo.

In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt

# Definir los nombres de las características y las clases
features_names = ['sepal length', 'sepal width', 'petal length', 'petal width']
target_names = ['setosa', 'versicolor', 'virginica']

# Graficar el primer árbol de decisión del modelo Random Forest
plt.figure(figsize=(12, 8))  # Ajustar el tamaño de la figura si es necesario
tree.plot_tree(tree_model,
               feature_names=features_names,
               class_names=target_names,
               filled=True,
               fontsize=10)
plt.show()



#Combinacion de multiples arboles de decisión mediante bosques aleatorios



**Random Forest** es un algoritmo de aprendizaje supervisado que combina varios **árboles de decisión** independientes para mejorar la precisión de las predicciones y reducir el riesgo de sobreajuste (**overfitting**).

### Proceso básico:
1. **Generación de submuestras**: Se crean múltiples árboles de decisión, cada uno entrenado con una submuestra diferente del conjunto de datos original, obtenida mediante **muestreo aleatorio con reemplazo** (bagging).
   
2. **Selección aleatoria de características**: En cada nodo de cada árbol, se selecciona aleatoriamente un subconjunto de características para decidir la mejor división. Esto introduce variabilidad entre los árboles y reduce la correlación entre ellos.

3. **Promedio o voto**: En problemas de regresión, las predicciones de los árboles se **promedian**. En problemas de clasificación, los árboles votan por la clase más frecuente, y el resultado final es el que obtiene más votos (**votación por mayoría**).

### Ventajas:
- **Robustez**: Al promediar los resultados de muchos árboles, se reduce el riesgo de sobreajuste.
- **Manejo de características**: Es capaz de manejar automáticamente características numéricas y categóricas, y no requiere normalización o estandarización.

En resumen, **Random Forest** es un algoritmo poderoso que utiliza múltiples árboles de decisión para generar predicciones más precisas y generalizables.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Crear y entrenar el modelo de Random Forest
# 'n_estimators=25' indica que se construirán 25 árboles de decisión.
# 'n_jobs=2' permite utilizar 2 núcleos de CPU para el entrenamiento en paralelo, lo que mejora la velocidad.
rf_model = RandomForestClassifier(n_estimators=25, random_state=1, n_jobs=2)

# Ajustar el modelo con el conjunto de entrenamiento.
# X_train_z son las características estandarizadas del conjunto de entrenamiento, y y_train son las etiquetas.
rf_model.fit(X_train_z, y_train)

plot_decision_region(rf_model, X_train_z, y_train)

# Configurar el título y las etiquetas del gráfico
plt.title('Random Forest - Regiones de Decisión')  # Título del gráfico
plt.xlabel('Característica 1 (Estandarizada)')  # Etiqueta del eje X
plt.ylabel('Característica 2 (Estandarizada)')  # Etiqueta del eje Y
plt.legend(loc='upper left')  # Colocar la leyenda en la parte superior izquierda
plt.grid(True)  # Mostrar la cuadrícula para una mejor visualización
plt.show()  # Mostrar la gráfica en pantalla


##Gradient Boosting

Es un método de aprendizaje supervisado que combina múltiples modelos débiles, generalmente árboles de decisión, para crear un modelo fuerte. La idea principal detrás del **boosting** es entrenar modelos de manera secuencial, donde cada modelo nuevo trata de corregir los errores cometidos por los modelos anteriores. En el caso de **Gradient Boosting**, esto se hace utilizando el **gradiente** de la función de pérdida para ajustar cada nuevo modelo.

### Funcionamiento de **Gradient Boosting**:
1. **Inicialización del modelo**:
   - El proceso comienza creando un modelo inicial, que puede ser un modelo muy simple, como un árbol de decisión con una sola división.
   - Este primer modelo genera predicciones iniciales.

2. **Calcular los residuos**:
   - En cada iteración, se calculan los **residuos** o **errores** (diferencia entre las predicciones actuales y los valores verdaderos).
   - Estos residuos indican qué tan lejos están las predicciones del modelo actual de los valores reales.

3. **Entrenamiento de un nuevo modelo**:
   - Un nuevo modelo se entrena para predecir los residuos de las iteraciones anteriores.
   - En lugar de predecir los valores directamente, el nuevo modelo predice las **correcciones** que deben aplicarse a las predicciones anteriores.

4. **Actualización de las predicciones**:
   - Las predicciones se actualizan sumando las correcciones predichas por el nuevo modelo.
   - Este proceso se repite, y cada nuevo modelo intenta mejorar las predicciones anteriores ajustándose a los errores que se cometieron.

5. **Tasa de aprendizaje (learning rate)**:
   - El ajuste de las predicciones no se hace de manera abrupta. Para controlar la contribución de cada modelo, se introduce un **learning rate** (tasa de aprendizaje) que reduce el impacto de cada nuevo modelo en la predicción final. Esto ayuda a evitar el sobreajuste.

### Ventajas de Gradient Boosting:
1. **Alta precisión**: Es uno de los algoritmos más precisos en muchos problemas de clasificación y regresión.
2. **Capacidad de manejo de datos no lineales**: Los árboles de decisión pueden capturar relaciones no lineales en los datos, y el boosting mejora estas capacidades.
3. **Flexibilidad**: Se puede aplicar a una variedad de funciones de pérdida (error cuadrático, log-loss, etc.), lo que lo hace adecuado tanto para problemas de clasificación como de regresión.

### Desventajas de Gradient Boosting:
1. **Riesgo de sobreajuste**: Debido a que el modelo se ajusta a los residuos en cada paso, puede sobreajustarse si no se controla adecuadamente (por ejemplo, si el número de árboles es muy alto o la profundidad de los árboles es grande).
2. **Coste computacional**: Dado que los modelos se entrenan secuencialmente, puede ser más lento en comparación con otros algoritmos como **Random Forest**, donde los árboles se entrenan en paralelo.



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Crear el modelo Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=30, learning_rate=0.1, max_depth=3, random_state=1)

# Entrenar el modelo con los datos de entrenamiento estandarizados
gb_model.fit(X_train_z, y_train)

# Hacer predicciones en el conjunto de prueba estandarizado
y_pred = gb_model.predict(X_test_z)

# Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo Gradient Boosting: {accuracy:.4f}')


**XGBoost** y **LightGBM** son dos poderosos algoritmos de **boosting**, una técnica de ensemble que combina varios modelos débiles (generalmente árboles de decisión) para crear un modelo fuerte que mejora el rendimiento de las predicciones. Ambos son muy eficientes en términos de velocidad y rendimiento, lo que los hace populares en competiciones y problemas de machine learning de gran escala.

A continuación, te explico cómo funcionan y cómo aplicarlos en un conjunto de datos.

---

### **XGBoost (Extreme Gradient Boosting)**

**XGBoost** es una implementación optimizada del algoritmo de **Gradient Boosting**. Este algoritmo construye árboles de decisión en secuencia, donde cada nuevo árbol intenta corregir los errores cometidos por los anteriores, enfocándose en las muestras mal clasificadas.

#### **Pasos para aplicar XGBoost**:
1. **Cargar y preparar los datos**.
2. **Dividir los datos en conjuntos de entrenamiento y prueba**.
3. **Entrenar el modelo de XGBoost** utilizando las características del conjunto de entrenamiento.
4. **Evaluar el modelo** en el conjunto de prueba.




In [ ]:

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Crear el modelo XGBoost
xgb_model = xgb.XGBClassifier(n_estimators=30, learning_rate=0.1, max_depth=3, random_state=1)

# Entrenar el modelo con los datos de entrenamiento
xgb_model.fit(X_train_z, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = xgb_model.predict(X_test_z)

# Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo XGBoost: {accuracy:.4f}')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

# Definir la función de visualización para las regiones de decisión
def plot_decision_region(clf, X, y, resolution=0.02):
    # Crear una paleta de colores y marcar diferentes clases
    markers = ('o', 's', '^', 'v')
    colors = ('#FF9999', '#66B2FF', '#99FF99')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # Seleccionar solo las primeras dos características para la visualización
    X = X.iloc[:, :2].values  # Convertir a numpy array con las dos primeras características

    # Crear los límites de los ejes
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))

    # Predecir las etiquetas para cada punto de la malla
    Z = clf.predict(np.c_[xx1.ravel(), xx2.ravel()])
    Z = Z.reshape(xx1.shape)

    # Crear la superficie de decisión
    plt.contourf(xx1, xx2, Z, alpha=0.3, cmap=cmap)

    # Ajustar límites de los ejes
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    # Graficar los puntos de datos
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1], alpha=0.8, color=colors[idx],
                    edgecolor='k', marker=markers[idx], label=f'Clase {cl}', s=100)

# Graficar las regiones de decisión usando el modelo ya entrenado (xgb_model)
plot_decision_region(xgb_model, X_train_z, y_train)

# Configurar el título y las etiquetas del gráfico
plt.title('XGBoost - Regiones de Decisión')
plt.xlabel('Característica 1 (Estandarizada)')
plt.ylabel('Característica 2 (Estandarizada)')
plt.legend(loc='upper left')
plt.grid(True)
plt.show()



#### **Principales parámetros de XGBoost**:
- **n_estimators**: Número de árboles.
- **learning_rate**: Tasa de aprendizaje para reducir el impacto de cada árbol.
- **max_depth**: Profundidad máxima de los árboles, controla el sobreajuste.
- **random_state**: Semilla para reproducibilidad.

---

### **LightGBM (Light Gradient Boosting Machine)**

**LightGBM** es una implementación eficiente de **Gradient Boosting**, diseñada para ser rápida y escalable, especialmente para grandes volúmenes de datos. A diferencia de XGBoost, LightGBM crece los árboles de decisión de forma **hoja a hoja** (leaf-wise), lo que mejora la precisión sin aumentar significativamente el tiempo de cómputo.

#### **Pasos para aplicar LightGBM**:
1. **Cargar y preparar los datos**.
2. **Dividir los datos en conjuntos de entrenamiento y prueba**.
3. **Entrenar el modelo LightGBM**.
4. **Evaluar el modelo**.


In [ ]:

import lightgbm as lgb

from sklearn.metrics import accuracy_score


# Crear el modelo LightGBM
lgb_model = lgb.LGBMClassifier(n_estimators=30, learning_rate=0.1, max_depth=3, random_state=1)

# Entrenar el modelo con los datos de entrenamiento
lgb_model.fit(X_train_z, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = lgb_model.predict(X_test_z)

# Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo LightGBM: {accuracy:.4f}')
warnings.filterwarnings('ignore')  # Ignorar todos los warnings

In [ ]:

# Definir la función de visualización para las regiones de decisión (igual que antes)
def plot_decision_region(clf, X, y, resolution=0.02):
    # Crear una paleta de colores y marcar diferentes clases
    markers = ('o', 's', '^', 'v')
    colors = ('#FF9999', '#66B2FF', '#99FF99')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # Seleccionar solo las primeras dos características para la visualización
    X = X.iloc[:, :2].values  # Convertir a numpy array con las dos primeras características

    # Crear los límites de los ejes
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))

    # Predecir las etiquetas para cada punto de la malla
    Z = clf.predict(np.c_[xx1.ravel(), xx2.ravel()])
    Z = Z.reshape(xx1.shape)

    # Crear la superficie de decisión
    plt.contourf(xx1, xx2, Z, alpha=0.3, cmap=cmap)

    # Ajustar límites de los ejes
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    # Graficar los puntos de datos
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1], alpha=0.8, color=colors[idx],
                    edgecolor='k', marker=markers[idx], label=f'Clase {cl}', s=100)

# Graficar las regiones de decisión usando el modelo ya entrenado (lgb_model)
plot_decision_region(lgb_model, X_train_z, y_train)

# Configurar el título y las etiquetas del gráfico
plt.title('LightGBM - Regiones de Decisión')
plt.xlabel('Característica 1 (Estandarizada)')
plt.ylabel('Característica 2 (Estandarizada)')
plt.legend(loc='upper left')
plt.grid(True)
plt.show()

#### **Principales parámetros de LightGBM**:
- **n_estimators**: Número de árboles.
- **learning_rate**: Tasa de aprendizaje.
- **max_depth**: Profundidad máxima de los árboles.
- **boosting_type**: Controla el tipo de boosting, puede ser 'gbdt' (por defecto) o 'dart'.
- **random_state**: Semilla para reproducibilidad.

---

### **Comparación entre XGBoost y LightGBM**:
- **XGBoost** es generalmente más flexible y ofrece más control en el ajuste de hiperparámetros, pero puede ser más lento en comparación con **LightGBM**.
- **LightGBM** es extremadamente eficiente para grandes conjuntos de datos y puede ser más rápido debido a su enfoque de crecimiento de árbol basado en hojas (leaf-wise), pero tiende a sobreajustar si no se controla adecuadamente.

Ambos algoritmos son muy poderosos y se pueden ajustar a problemas de clasificación y regresión. Puedes probar ambos en tu conjunto de datos y elegir el que te ofrezca mejor rendimiento en términos de precisión y velocidad.

# Conclusiones

En resumen, todos los clasificadores estudiados (SVM, Árboles de Decisión, Random Forest, XGBoost, y LightGBM) tienen características y fortalezas particulares que los hacen más adecuados para diferentes tipos de problemas. **SVM** es eficaz cuando se trata de problemas con fronteras de decisión claras y es particularmente útil en conjuntos de datos de alta dimensionalidad, aunque su rendimiento puede disminuir con grandes volúmenes de datos. Los **Árboles de Decisión** son fáciles de interpretar y visualmente intuitivos, pero tienden a sobreajustarse fácilmente si no se controlan adecuadamente.

**Random Forest**, al combinar varios árboles de decisión, mejora la robustez y reduce el riesgo de sobreajuste, además de ser eficaz en problemas donde se requiere interpretabilidad y estabilidad. Por otro lado, **XGBoost** y **LightGBM** sobresalen en escenarios donde se necesita alta precisión y eficiencia en grandes conjuntos de datos, utilizando técnicas avanzadas de **boosting** para corregir los errores iterativamente. Si bien todos los modelos ofrecen buenos resultados, **XGBoost** y **LightGBM** son preferidos en problemas más complejos, mientras que **Random Forest** y **SVM** son útiles para implementaciones más sencillas o cuando las características del problema favorecen estos enfoques.